In [1]:
%run iteration_01_0_imports.ipynb

In [2]:
%run iteration_01_0_parameters.ipynb

In [3]:
%run iteration_01_0_utils.ipynb

In [4]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

# Prétraitement

## positif_global_std

In [5]:
IMG_IN_COLOR = 1
def positive_global_std(path_img: str) -> np.array:
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    mean = img_arr.mean()
    std  = img_arr.std()
    img_arr_std = (img_arr - mean) / std
    
    # clip les valeurs
    img_arr_std = np.clip(img_arr_std, -1.0, 1.0)
    
    # shift to 0--1
    img_arr_std = (img_arr_std + 1.0) / 2.0
    
    return img_arr_std

* Voir avec le prof la pertinence de clip les valeurs extremes -1 et 1

In [6]:
NBR_SAMPLE

2000

In [7]:
df_path_img_healthy_sample = df_path_img_healthy.sample(n = int(NBR_SAMPLE/2))
df_path_img_sick_sample    = df_path_img_sick.sample(n = int(NBR_SAMPLE/2))

In [8]:
PATH_DATA_SAMPLE

'../data/sample'

In [9]:
DATE_TIME = "2021-11-18_20-39-00"

def create_sample(df_path_img:str, img_type:str, path_sample:str, date_time:str, clean_dir:bool):
    path_sample_time = path_sample+"/"+date_time

    create_dir_hierarchy(path_sample_time)
    if clean_dir:
        file_list = glob.glob(path_sample_time+"/"+"*.*")
        for f in file_list:
            os.remove(f)
    
    for path_img , _ in zip(df_path_img.path_img, tqdm(range(len(df_path_img.path_img)))):
        
        _, img_name, _ = split_file_info(path_img)
        
        img_arr = positive_global_std(path_img)
        # padding
        img_arr_padded = np.zeros((IMG_SIZE, IMG_SIZE, 3))
        img_arr_padded[:img_arr.shape[0],:img_arr.shape[1],:] = img_arr
        
        pickel_save(img_arr_padded, path_sample_time + "/" + str(img_type).upper() + "_" + img_name + ".pkl")
        

In [10]:
create_sample(df_path_img_healthy_sample, HEALTHY, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=True)

100%|██████████████████████████████████████████████████████████████████████████████▉| 999/1000 [01:22<00:00, 12.16it/s]


In [11]:
create_sample(df_path_img_sick_sample, SICK, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=False)

100%|██████████████████████████████████████████████████████████████████████████████▉| 999/1000 [01:57<00:00,  8.51it/s]


In [21]:
df_path_img_sample = pd.concat([df_path_img_healthy_sample, df_path_img_sick_sample], ignore_index=True)
df_path_img_not_in_sample = pd.DataFrame([i for i in df_path_img.path_img.values if not i in df_path_img_sample.path_img.values], 
                 columns=df_path_img_sample.columns)

In [23]:
df_path_img_not_in_sample.shape

(25558, 1)

In [24]:
df_path_img_not_in_sample.head()

,path_img
0,../cell_images/Parasitized\C100P61ThinF_IMG_20...
1,../cell_images/Parasitized\C100P61ThinF_IMG_20...
2,../cell_images/Parasitized\C100P61ThinF_IMG_20...
3,../cell_images/Parasitized\C100P61ThinF_IMG_20...
4,../cell_images/Parasitized\C100P61ThinF_IMG_20...


In [27]:
df_path_img_not_in_sample.sample(1).values

array([['../cell_images/Parasitized\\C59P20thinF_IMG_20150803_112858_cell_163.png']],
      dtype=object)